In [1]:
import sys
import time
import exifread
import cupy as cp
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

In [4]:
def image_import():
    image_path = "image.tif"
    image = Image.open(image_path)
    
    exif_file = open(str(image_path), 'rb')
    tags = exifread.process_file(exif_file, strict = False)
    if "Image Tag 0x8546" in tags.keys():
        print("Finding SEM information in image file successful.")

        # SEM information metadata is a very long binary string and includes data 
        #  that seem to be non decodable ASCII / UTF-8. The information needed is 
        #  in the first 100 characters, so everything after that is ignored.
        stuff = tags["Image Tag 0x8546"]
        stuff_value = stuff.values
        stuff_value = stuff_value[0:100].decode('utf-8')
        stuff_value_list = stuff_value.splitlines()
        
        pixelsize = stuff_value_list[3]
        magnification = stuff_value_list[4]
    else:
        print("Finding SEM information in image file failed.")
        sys.exit(0)
    
    return image, image_path, float(magnification), float(pixelsize)
    
def image_fft_python(image):
    image_array = np.asarray(image)
    return np.fft.rfft2(image_array)

def image_fft_gpu(image):
    image_array = cp.asarray(image)
    return cp.fft.rfft2(image_array)

def main():
    print("Program starts.")
    image, image_path, magnification, pixelsize = image_import()
    
    time_start = time.time()
    image_array_fft = image_fft_python(image)
    time_stop = time.time()
    print("Time taken for execution of the FFT in Python is: ", 
            time_stop - time_start, "s")
    
    time_start = time.time()
    image_array_fft = image_fft_gpu(image)
    time_stop = time.time()
    print("Time taken for first execution of the FFT on the GPU is: ", 
            time_stop - time_start, "s")

    time_start = time.time()
    image_array_fft = image_fft_gpu(image)
    time_stop = time.time()
    print("Time taken for second execution of the FFT on the GPU is: ", 
            time_stop - time_start, "s")

    return 1

In [5]:
main()

Possibly corrupted field Tag 0x8546 in Image IFD
Program starts.
Finding SEM information in image file successful.
Time taken for execution of the FFT in Python is:  0.01097249984741211 s
Time taken for first execution of the FFT on the GPU is:  1.2905497550964355 s
Time taken for second execution of the FFT on the GPU is:  0.003987789154052734 s


1